In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

In [10]:
# Captura del tema de interés de la persona
interest = input("¿Cuál es tu tema de interés: ")

# Opciones para el navegador
chrome_options = Options()
chrome_options.add_argument("--star-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--window-size=%s" % "1920,1080")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Navegador
driver = webdriver.Chrome(
    service = Service(ChromeDriverManager().install()),
    options = chrome_options
)
driver.maximize_window()
time.sleep(1) # Se deja 1 segundo para que la página logré cargar

# Página que se está consultando
driver.get("https://co.computrabajo.com/")


# Esta parte es para hacer consultas automáticas

# Tema de interés a consultar
WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((
        By.CSS_SELECTOR, 'input#prof-cat-search-input'
    ))
).send_keys(interest)

# Acción para hacer clic
WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((
        By.CSS_SELECTOR, 'button#search-button'
    ))
).click()

# Estructura de las ofertas
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((
        By.XPATH, '/html/body/main/div[4]/div[2]/div[1]/div[2]'
    ))
)

# Selectores especiales
html_list = driver.find_element(By.CSS_SELECTOR, "div#p_ofertas")
next_page = driver.find_element(By.CSS_SELECTOR, "span.b_next.buildLink.cp")
items = html_list.find_elements(By.TAG_NAME, "article")

# Variables para DataFrame
titles = []
companies_name = []
companies_link = []
how_longs = []
short_descriptions = []

# Ciclo para captura de los datos
for item in items:
    text = item.get_attribute('innerHTML')
    soup = BeautifulSoup(text, 'html.parser')

    # Valores a recuperar    
    company = soup.select("a.fc_base.hover.it-blank")
    href = soup.select("a.fc_base.hover.it-blank")
    
    titles.append(soup.find_all("h1")[0].string)
    short_descriptions.append(soup.select("p.fc_aux.t_word_wrap.mb10.hide_m")[0].text)
    how_longs.append(soup.select("p.fs13.fc_aux")[0].text)


    if company and href: 
        companies_name.append(company[0].text)
        companies_link.append(href[0].get('href'))
    else: 
        companies_name.append("Sin empresa registrada...")
        companies_link.append("Sin link de empresa...")

# Creación del diccionario    
data_jobs = {
    "titles": titles,
    "companies_name": companies_name,
    "companies_link": companies_link,
    "how_longs": how_longs,
    "short_descriptions": short_descriptions
}

# Creación del archivo csv para análisis
df = pd.DataFrame(data_jobs)
df.to_csv("WebScrapingCompuTrabajo.csv")

# Título que se capturó
print("Web Scraping finalizado a: " + driver.title)
driver.close() # Esto es para cerrar la ventana del navegador

¿Cuál es tu tema de interés: Laravel
Web Scraping a: Trabajo de laravel | Portal de Empleo Computrabajo
